In [1]:
########################################
# ライブラリ
########################################

import jax
import jax.numpy as jnp

from flax import linen as nn
from flax.training import train_state
import optax

from tqdm import trange as tqdm_range
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
########################################
# モデルの定義
########################################

class ANN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=100)(x)
        x = nn.relu(x)
        x = nn.Dense(features=3)(x)
        x = nn.softmax(x)
        return x

In [3]:
########################################
# 損失関数
########################################

@jax.jit
def loss_function(params, X, y):
    # 予測値計算
    predict = ann.apply({'params': params}, X)
    # 損失を計算
    loss = jnp.mean(-jnp.sum(y * jnp.log(predict), axis=1))
    return loss

In [4]:
########################################
# ミニバッチ学習
########################################

@jax.jit
def train_batch(batch_idx, state):
    # ミニバッチを抽出
    target_indices = jax.lax.dynamic_slice_in_dim(train_indices, (batch_idx*batch_length), batch_length)
    X, y = X_train[target_indices], y_train[target_indices]
    # 損失と勾配を計算
    loss, grads = jax.value_and_grad(loss_function)(state.params, X, y)
    # 更新
    state = state.apply_gradients(grads=grads)
    return state

In [5]:
########################################
# データセットの読み込み
########################################

iris_dataset = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_dataset['data'], iris_dataset['target'], test_size=0.25,  random_state=0)
X_train, X_test, y_train, y_test = jax.device_put(X_train), jax.device_put(X_test), jax.device_put(y_train), jax.device_put(y_test)
y_train = jnp.eye(3)[y_train]
y_test = jnp.eye(3)[y_test]

In [6]:
########################################
# 学習してみる
########################################

ann = ANN()
params = ann.init(jax.random.PRNGKey(0), jnp.ones([1, 4]))['params']

# SGDを定義
learning_rate, momentum = 0.01, 0.01
tx = optax.sgd(learning_rate, momentum)

# パラメータの管理
state = train_state.TrainState.create(apply_fn=ann.apply, params=params, tx=tx)

# エポック数
nums_epoch = 100
# バッチサイズ
batch_size = 5

# バッチ数
batch_length = X_train.shape[0] // batch_size

# nums_epoch回 学習する
for epoch_idx in range(nums_epoch):
    
    # 訓練データのインデックスをシャッフル
    train_indices = jax.random.permutation(jax.random.PRNGKey(epoch_idx+1), X_train.shape[0])
    # ミニバッチ学習で更新
    state = jax.lax.fori_loop(0, batch_length, train_batch, state)
    
    # 誤差の確認
    print(
        "訓練誤差:",
        '{:.3f}'.format(loss_function(state.params, X_train, y_train)),
        "汎化誤差:",
        '{:.3f}'.format(loss_function(state.params, X_test, y_test)),
        f"【 Epoch: {epoch_idx} / {nums_epoch} 】"
    )

訓練誤差: 0.878 汎化誤差: 1.171 【 Epoch: 0 / 100 】
訓練誤差: 0.729 汎化誤差: 1.004 【 Epoch: 1 / 100 】
訓練誤差: 0.650 汎化誤差: 0.916 【 Epoch: 2 / 100 】
訓練誤差: 0.589 汎化誤差: 0.848 【 Epoch: 3 / 100 】
訓練誤差: 0.535 汎化誤差: 0.787 【 Epoch: 4 / 100 】
訓練誤差: 0.485 汎化誤差: 0.728 【 Epoch: 5 / 100 】
訓練誤差: 0.436 汎化誤差: 0.669 【 Epoch: 6 / 100 】
訓練誤差: 0.389 汎化誤差: 0.609 【 Epoch: 7 / 100 】
訓練誤差: 0.342 汎化誤差: 0.546 【 Epoch: 8 / 100 】
訓練誤差: 0.297 汎化誤差: 0.481 【 Epoch: 9 / 100 】
訓練誤差: 0.255 汎化誤差: 0.413 【 Epoch: 10 / 100 】
訓練誤差: 0.219 汎化誤差: 0.346 【 Epoch: 11 / 100 】
訓練誤差: 0.196 汎化誤差: 0.292 【 Epoch: 12 / 100 】
訓練誤差: 0.186 汎化誤差: 0.258 【 Epoch: 13 / 100 】
訓練誤差: 0.181 汎化誤差: 0.239 【 Epoch: 14 / 100 】
訓練誤差: 0.176 汎化誤差: 0.228 【 Epoch: 15 / 100 】
訓練誤差: 0.171 汎化誤差: 0.221 【 Epoch: 16 / 100 】
訓練誤差: 0.166 汎化誤差: 0.216 【 Epoch: 17 / 100 】
訓練誤差: 0.161 汎化誤差: 0.212 【 Epoch: 18 / 100 】
訓練誤差: 0.157 汎化誤差: 0.209 【 Epoch: 19 / 100 】
訓練誤差: 0.152 汎化誤差: 0.206 【 Epoch: 20 / 100 】
訓練誤差: 0.149 汎化誤差: 0.204 【 Epoch: 21 / 100 】
訓練誤差: 0.145 汎化誤差: 0.201 【 Epoch: 22 / 100 